In [1]:
#%%
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten,Dropout,Conv2D,LSTM, LeakyReLU, MaxPooling2D
import keras
from datetime import datetime
from pathlib import Path
from scipy import signal
from os import walk
from keras.preprocessing import sequence
from scipy.io import wavfile
import glob
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split

import tensorflow as tf

from multiprocessing.dummy import Pool as ThreadPool

pool=ThreadPool(24)


In [2]:
log_dir = "/tf/logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
#%tensorboard

In [3]:
# Load wavefile
def readwav(file:str):
    filepath = Path(file).absolute()
    samplerate, data = wavfile.read((filepath))
    # print(f"samplerate = {samplerate}")
    return data,samplerate

In [4]:
def getTiming(data:np.ndarray,samplerate:int):
    length = data.shape[0] / samplerate
    return np.arange(0,length,1/samplerate)



In [5]:
def filterSignal(data:np.ndarray,t:np.ndarray,plot:bool=True):
    ## normalize input
    sig = data/np.amax(data)
    sos = signal.butter(1, [.2,3], 'bp', fs=1000, output='sos')
    filtered_heart = signal.sosfilt(sos, sig)
    norm_heart = filtered_heart/np.amax(filtered_heart)
    ## Removing noise
    norm_heart = signal.signaltools.wiener(filtered_heart,300)
    if plot:
        _, (ax1, ax2,ax3) = plt.subplots(3, 1, sharex=True)
        ax1.plot(t, sig)
        ax1.set_title('Original Heart Rate Signal')
        ax2.plot(t, norm_heart)
        ax2.set_title('After Bandpass filter')
        ax3.plot(t, norm_heart)
        ax3.set_title('After Noise Filter')
        ax3.set_xlabel('Time [seconds]')
        plt.tight_layout()
        plt.show()
    return norm_heart
    

In [6]:
def generateSignal(file:str,plot:bool=False,loglevel:str=None):
    data,samplerate = readwav(file)
    t = getTiming(data,samplerate)
    return t,filterSignal(data,t,plot),samplerate

In [7]:
trainingFolder = "./heartbeats/classifications"
trainingpath = Path(trainingFolder)
paths = [Path(dir[0]) for dir in walk(trainingpath)][1:]


In [8]:
train_data = []
# test_data = []
plot = False
def processFiles(indexedWave,classification,trainIndex):
    wav,index = indexedWave
    t,d,_ = generateSignal(wav,plot)
    # if(index < trainIndex):
    train_data.append([d,t,classification])
    # else:
        # test_data.append([d,t,classification])


def get_training_data(path:Path):   
    classification = path.name
    wavList = glob.glob(str(path.joinpath("*.wav")))
    trainIndex=int(math.ceil(len(wavList)*.8)) # use 80% of data for training
    pool.map(lambda x: processFiles(x,classification,trainIndex),zip(wavList,range(0,len(wavList))) )
    # for wav in zip(wavList,range(0,len(wavList))):
    #     processFiles(wav,classification,trainIndex)




In [9]:
# Load Data 
#Loading data from this many files is intensive, speeding up w/ multithreading

for path in paths:
    get_training_data(path)
#train_data=np.array(train_data)
#test_data=np.array(test_data)



In [10]:
sig,time,classification = zip(*train_data)
getMaxLength = lambda list: max([len(item) for item in list])


In [11]:
# data processing
x = np.stack([sequence.pad_sequences(dat,maxlen=getMaxLength(dat),dtype='float64') for dat in [sig,time]],axis=1)
print(np.shape(x))
## integer encode
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(np.array(classification))


(176, 2, 396900)


In [12]:
x = x.reshape(x.shape[0],396900,2)
y=tf.keras.utils.to_categorical(y,num_classes=5)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

In [13]:
# model=Sequential()
# model.add(Dense(16, input_shape=(x_train.shape[1],1), activation="relu"))
# model.add(Dense(8, activation='tanh'))
# model.add(Dense(5, activation='sigmoid'))
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])



In [2]:
def make_model(input_shape):
    input_layer = keras.layers.Input(input_shape)

    conv1 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(input_layer)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.ReLU()(conv1)

    conv2 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv1)
    conv2 = keras.layers.BatchNormalization()(conv2)
    conv2 = keras.layers.ReLU()(conv2)

    conv3 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv2)
    conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.ReLU()(conv3)

    gap = keras.layers.GlobalAveragePooling1D()(conv3)

    output_layer = keras.layers.Dense(5, activation="softmax")(gap)

    return keras.models.Model(inputs=input_layer, outputs=output_layer)


model = make_model(input_shape=x_train.shape[1:])
tf.keras.utils.plot_model(model, show_shapes=True)

NameError: name 'x_train' is not defined

In [14]:

# model = Sequential()
# # model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
# # model.add(MaxPooling1D(pool_size=2))
# model.add(LSTM(100,return_sequences=False))

# # model.add(LSTM(400,return_sequences=False,recurrent_activation='selu',kernel_initializer='lecun_normal', input_shape=(x_train.shape[1],1)))

# model.add(Dense(5, activation='softmax'))
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


2021-11-12 23:17:31.867792: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2021-11-12 23:17:31.877154: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2021-11-12 23:17:31.877485: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2021-11-12 23:17:31.878369: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate

In [15]:
model.fit(x_train, y_train, epochs=10, batch_size=8)
model.summary()


2021-11-12 23:17:34.642641: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 501681600 exceeds 10% of free system memory.
2021-11-12 23:17:35.779754: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 501681600 exceeds 10% of free system memory.


Epoch 1/10


2021-11-12 23:17:38.014219: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 25401600 exceeds 10% of free system memory.
2021-11-12 23:17:38.014285: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 25401600 exceeds 10% of free system memory.
2021-11-12 23:17:38.014314: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 25401600 exceeds 10% of free system memory.
2021-11-12 23:17:40.378161: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8100
2021-11-12 23:19:14.881659: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


17/20 [========================>.....] - ETA: 10:47 - loss: 1.6375 - accuracy: 0.2647

In [ ]:
predictions = model.predict(test_set)

Error: Session cannot generate requests

In [ ]:
scores = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))